In [17]:
%reload_ext autoreload
%autoreload 2

import os
import sys

import numpy as np

import py21cmfast as p21c

sys.path.append("..")
from dm21cm.dm_params import DMParams
from dm21cm.evolve import evolve

INFO:root:Using 21cmFAST version 0.1.dev1578+g6f96f89.d20230224


## config

In [2]:
print(os.cpu_count())
n_thread = 32

48


In [3]:
# check cached runs
l = os.listdir(os.environ['P21C_CACHE_DIR'])
l.sort()
[print(i) for i in l];

base
base_dhinit
base_zfine
compdh_base
compdh_base_dhinit
compdh_base_finestep
phph_dhinit
phph_dhinit_fine
phph_dhinit_s8zero
phph_dhinit_s8zero_fine
xray_bath_test
xray_test
xray_test_all_to_unif


In [4]:
os.environ['DM21CM_DATA_DIR'] = '/data/submit/yitians/dm21cm/DM21cm/data'

## run

In [19]:
evolve(
    run_name = 'phph_dhinit_s8zero_fine',
    run_mode = 'bath',
    z_start = 44,
    z_end = 6,
    zplusone_step_factor = 1.01,
    dm_params = DMParams(
        mode = 'swave',
        primary = 'phot_delta',
        m_DM = 1e10,
        sigmav = 1e-23,
    ),
    struct_boost_model = 'erfc 1e-3',
    enable_elec = False,
    dhinit_list = ['phot', 'T_k', 'x_e'],
    dhtf_version = '230629',
    
    p21c_initial_conditions = p21c.initial_conditions(
        user_params = p21c.UserParams(
            HII_DIM = 50, # [1] | base: 50
            BOX_LEN = 50, # [p-Mpc] | base: 50
            N_THREADS = 32
        ),
        cosmo_params = p21c.CosmoParams(
            OMm = 0.32,
            OMb = 0.049,
            POWER_INDEX = 0.96,
            SIGMA_8 = 0.0001, # base 0.83
            hlittle = 0.67
        ),
        random_seed = 54321,
        write = True,
    ),
    
    rerun_DH = False,
    clear_cache = True,
    force_reload_tf = True,
    
    use_tqdm = True,
    save_slices = True,
)

INFO:root:Loaded photon propagation transfer function.
INFO:root:Loaded photon scattering transfer function.
INFO:root:Loaded photon deposition transfer function.
100%|██████████| 187/187 [18:56<00:00,  6.07s/it]

input used 650.9241 s
p21c used 482.2711 s
